In [ ]:
#Şimdi haber kategori sınıflandırmada bir önceki örnekte TF-IDF yöntemiyle çözmeye
#çalıştığımız sınıflandırma işlemindeki problemi BERT modeli ile çözmeye çalışalım


#klasik yöntem kelime bazlı cümlelere baktığı için  setteki sınıflandırmayı doğru
#yapsa da daha sonra girilen girdideki kelimeler eğitim setinde az geçtiyse yanlış
#sınıflandırabiliyordu. Bunun sebebi cümle bazlı değil kelime bazlı
#bakmasıydı bu durumu BERT ile çözebiliriz.
"""
BERT, Google tarafından geliştirilen ve metnin bağlamını anlamaya odaklanan
 bir derin öğrenme modelidir."Bidirectional" yani çift yönlü çalışır → cümledeki
 bir kelimenin hem öncesine hem sonrasına bakarak anlam çıkarır.

"""


"""
| Özellik             | TF-IDF                     | BERT                        |
| ------------------- | -------------------------- | --------------------------- |
| Çalışma mantığı     | Kelime frekansına bakar    | Anlama, bağlama bakar       |
| Bağlam bilgisi      | ❌ Yok                      | ✅ Var                       |
| Anlam çözümü        | ❌ Yapamaz                  | ✅ Yapar                     |
| “Transfer” kelimesi | Spor mu? Sağlık mı? Bilmez | Cümledeki yerine göre anlar |
| Yeni cümleler       | Şaşırabilir                | Genelde doğru sınıflandırır |


BERT Ne Öğrenir?
Kelimelerin sadece kendisini değil, çevresindeki anlamı

“bank” kelimesinin “nehrin kenarı” mı, “para kurumu” mu olduğunu, cümleye göre anlar.

Örneğin:

“Bankta oturuyorum” → oturulan yer

“Bank kredi verdi” → finans kurumu
"""

'\n| Özellik             | TF-IDF                     | BERT                        |\n| ------------------- | -------------------------- | --------------------------- |\n| Çalışma mantığı     | Kelime frekansına bakar    | Anlama, bağlama bakar       |\n| Bağlam bilgisi      | ❌ Yok                      | ✅ Var                       |\n| Anlam çözümü        | ❌ Yapamaz                  | ✅ Yapar                     |\n| “Transfer” kelimesi | Spor mu? Sağlık mı? Bilmez | Cümledeki yerine göre anlar |\n| Yeni cümleler       | Şaşırabilir                | Genelde doğru sınıflandırır |\n\n\nBERT Ne Öğrenir?\nKelimelerin sadece kendisini değil, çevresindeki anlamı\n\n“bank” kelimesinin “nehrin kenarı” mı, “para kurumu” mu olduğunu, cümleye göre anlar.\n\nÖrneğin:\n\n“Bankta oturuyorum” → oturulan yer\n\n“Bank kredi verdi” → finans kurumu\n'

In [ ]:
from google.colab import files
uploaded = files.upload()


Saving TurkishHeadlines.csv to TurkishHeadlines.csv


In [ ]:
import pandas as pd

df = pd.read_csv("TurkishHeadlines.csv")
df.head()


,HABERLER,ETIKET
0,TÜİK verilerine göre sanayi ciro endeksi Ağust...,Ekonomi
1,Piyasa güne eksi rezervde başladı,Ekonomi
2,"Citigroup, Deutsche Bank ve HSBC Libor manipül...",Ekonomi
3,Gelişen piyasa yatırımcılarını en fazla 'Fed' ...,Ekonomi
4,Bitcoin fiyatında yükseliş hız kesmiyor,Ekonomi


In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder


df.columns = ['haber', 'etiket']
#sütun adlarını küçültme sadeleştirme

label_encode= LabelEncoder()
df['label']=label_encode.fit_transform(df['etiket'])

#etiketleri sayısal değerlere döüştürme


num_labels = df['label'].nunique()
print(f"Sınıf Sayısı: {num_labels}")
print(label_encode.classes_)
#Sınıf bilgisi

train_texts, test_texts, train_labels, test_labels=train_test_split(
    df['haber'].tolist(),df['label'].tolist(),
    test_size=0.2,random_state=42, stratify=df['label']

)

#%80'e %20 eğitim test verisi ayrımı yapıldı

Sınıf Sayısı: 7
['Ekonomi' 'Magazin' 'Sağlık' 'Siyaset' 'Spor' 'Teknoloji' 'Yaşam']


In [ ]:
#Hugging Faze Tokenizer ile metinleri modele hazırlama

!pip install transformers --quiet




In [ ]:
from transformers import BertTokenizer

tokenizer=BertTokenizer.from_pretrained("dbmdz/bert-base-turkish-cased")
#Türkçe BERT tokenizer yükleme

train_encodings= tokenizer(
    train_texts, truncation=True, padding=True, max_length=64

    )#eğitim ve test metinlerini tokenize etme


test_encodings = tokenizer(
    test_texts, truncation=True, padding=True, max_length=64
)


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


In [ ]:
"""
| Parametre         | Ne işe yarar?                                          |
| ----------------- | ------------------------------------------------------ |
| `truncation=True` | Uzun cümleleri keser (maksimum 64 token alır)          |
| `padding=True`    | Kısa cümlelerin uzunluğunu eşitler (64'e tamamlar)     |
| `max_length=64`   | Maksimum token sayısı (haber başlıkları için idealdir) |

"""

"\n| Parametre         | Ne işe yarar?                                          |\n| ----------------- | ------------------------------------------------------ |\n| `truncation=True` | Uzun cümleleri keser (maksimum 64 token alır)          |\n| `padding=True`    | Kısa cümlelerin uzunluğunu eşitler (64'e tamamlar)     |\n| `max_length=64`   | Maksimum token sayısı (haber başlıkları için idealdir) |\n\n"

In [ ]:
#Tesnor Formatına çevirme Dataset oluşturma

import torch

from torch.utils.data import Dataset

class NewDataset(Dataset):

  def __init__(self,encodings,labels):
    self.encodings=encodings
    self.labels=labels


  def __len__(self):
    return len(self.labels)

  def __getitem__(self,idx):

    item={key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
    item["labels"] = torch.tensor(self.labels[idx])
    return item


train_dataset=NewDataset(train_encodings,train_labels)
test_dataset=NewDataset(test_encodings, test_labels)


In [ ]:
#Model Yükleme

from transformers import BertForSequenceClassification

model = BertForSequenceClassification.from_pretrained(
    "dbmdz/bert-base-turkish-cased",
    num_labels=7
)

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at dbmdz/bert-base-turkish-cased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
!pip install --upgrade transformers
!pip install -U transformers



In [ ]:
import os
os.environ["WANDB_DISABLED"] = "true"



In [ ]:
#Trainer ve Eğitim


from transformers import Trainer,TrainingArguments


training_args = TrainingArguments(
    output_dir="./results",
    num_train_epochs=3,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=64,
    logging_dir="./logs",
    logging_steps=10
)



trainer=Trainer(

   model=model,
   args=training_args,
   train_dataset=train_dataset,
   eval_dataset=test_dataset,
)

Using the `WANDB_DISABLED` environment variable is deprecated and will be removed in v5. Use the --report_to flag to control the integrations used for logging result (for instance --report_to none).


In [ ]:
trainer.train()


/usr/local/lib/python3.11/dist-packages/torch/nn/modules/module.py:1750: FutureWarning: `encoder_attention_mask` is deprecated and will be removed in version 4.55.0 for `BertSdpaSelfAttention.forward`.
  return forward_call(*args, **kwargs)


Step,Training Loss
10,1.901200
20,1.569800
30,0.888200
40,0.278000
50,0.120200
60,0.040500
70,0.105600
80,0.063800
90,0.284200
100,0.019100


/usr/local/lib/python3.11/dist-packages/torch/nn/modules/module.py:1750: FutureWarning: `encoder_attention_mask` is deprecated and will be removed in version 4.55.0 for `BertSdpaSelfAttention.forward`.
  return forward_call(*args, **kwargs)


TrainOutput(global_step=630, training_loss=0.09622322217014337, metrics={'train_runtime': 7462.097, 'train_samples_per_second': 1.351, 'train_steps_per_second': 0.084, 'total_flos': 331534812672000.0, 'train_loss': 0.09622322217014337, 'epoch': 3.0})

In [ ]:
trainer.save_model("bert_modelim")  # modeli kaydet
tokenizer.save_pretrained("bert_modelim")  # tokenizer'ı da kaydet


('bert_modelim/tokenizer_config.json',
 'bert_modelim/special_tokens_map.json',
 'bert_modelim/vocab.txt',
 'bert_modelim/added_tokens.json')

In [ ]:
from transformers import BertForSequenceClassification, BertTokenizerFast
import torch

#model ve tokenizer ı yükleme
model = BertForSequenceClassification.from_pretrained("bert_modelim")
tokenizer = BertTokenizer.from_pretrained("bert_modelim")
model.eval()  #tahmin moduna alma

BertForSequenceClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(32000, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0-11): 12 x BertLayer(
          (attention): BertAttention(
            (self): BertSdpaSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e

In [ ]:
#YENİ GİRDİLERLE TAHMİN

def bert_tahmin(metin):
  inputs = tokenizer(metin, return_tensors="pt",truncation=True, padding=True)
  with torch.no_grad():
    outputs = model(**inputs)

  logits = outputs.logits
  tahmin=torch.argmax(logits,dim=1).item()

  return tahmin # 0 veya 1


etiket_map = {
    0: "Ekonomi",
    1: "Magazin",
    2: "Sağlık",
    3: "Siyaset",
    4: "Spor",
    5: "Teknoloji",
    6: "Yaşam"
}#örnek
#etikerleri eşleştirme

metin="Fenerbahçe trasnfer bombası patlattı"

print("Kategori:", etiket_map[bert_tahmin(metin)])

Kategori: Spor


In [ ]:
def kullanicidan_haber_kategorisi_al():
    while True:
        baslik = input("Haber başlığı girin (çıkmak için q): ")
        if baslik.lower() == "q":
            print("Çıkılıyor...")
            break
        inputs = tokenizer(baslik, return_tensors="pt", truncation=True, padding=True)
        with torch.no_grad():
            outputs = model(**inputs)
        tahmin_id = torch.argmax(outputs.logits, dim=1).item()
        kategori = etiket_map[tahmin_id]
        print(f"📢 Tahmin edilen kategori: {kategori}\n")


In [ ]:
kullanicidan_haber_kategorisi_al()


Haber başlığı girin (çıkmak için q): Sağlık Bakanlığı grip aşısı uyarısı yaptı
📢 Tahmin edilen kategori: Sağlık

Haber başlığı girin (çıkmak için q): Ünlü oyuncu boşanma davası açtı
📢 Tahmin edilen kategori: Magazin

Haber başlığı girin (çıkmak için q): Ev dekorasyonunda 2025 trendleri neler?
📢 Tahmin edilen kategori: Teknoloji

Haber başlığı girin (çıkmak için q): Cumhurbaşkanı seçim tarihini açıkladı
📢 Tahmin edilen kategori: Siyaset

Haber başlığı girin (çıkmak için q): Altın fiyatları rekor kırdı
📢 Tahmin edilen kategori: Ekonomi

Haber başlığı girin (çıkmak için q): q
Çıkılıyor...


In [ ]:
!zip -r bert_modelim.zip bert_modelim  #modeli zip oalrak kaydetme


  adding: bert_modelim/ (stored 0%)
  adding: bert_modelim/training_args.bin (deflated 52%)
  adding: bert_modelim/vocab.txt (deflated 53%)
  adding: bert_modelim/config.json (deflated 56%)
  adding: bert_modelim/model.safetensors (deflated 7%)
  adding: bert_modelim/tokenizer_config.json (deflated 75%)
  adding: bert_modelim/special_tokens_map.json (deflated 42%)


In [ ]:
from google.colab import files
files.download("bert_modelim.zip")
#zipi indirme

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>